In [ ]:
import pandas as pd
import numpy as np

import apimoex
import requests


import matplotlib.pyplot as plt
from numpy.typing import ArrayLike

In [1]:
stock_names = ['RASP', 'GAZP', 'DSKY', 'SBER', 'KMAZ', 'RUAL']
start_date = '2023-01-01'
end_date = '2023-05-05'

In [2]:
def load_data(stock_names=stock_names, start=start_date, end=end_date):
    df = pd.DataFrame()
    with requests.Session() as session:
        for stock in stock_names:
            data = apimoex.get_board_history(session, stock, start=start, end=end, columns=['CLOSE', 'TRADEDATE'])
            df_stock = pd.DataFrame(data)
            df_stock.set_index('TRADEDATE', inplace=True)
            df[stock] = df_stock
    return df


def prepare_r(df_stocks):
    return (df_stocks.shift(-1) / df_stocks).dropna() 



In [ ]:
def get_grad(r, x):
    return -r / np.dot(r, x)


def _unsafe_simplex_projection(s: ArrayLike, norm_constraint: float) -> ArrayLike:
    """Находит проекцию на симплекс."""
    u = np.sort(s)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, len(u) + 1) > (cssv - norm_constraint))[0][-1]
    theta = (cssv[rho] - norm_constraint) / (rho + 1.0)
    return np.maximum(s - theta, 0)


def simplex_projection(s: ArrayLike):
    """Возвращает проекцию на единичный симплекс."""
    return s if np.sum(s) == 1 else _unsafe_simplex_projection(s, 1.0)

In [ ]:
def get_this_score(r, x):
    return -np.log(np.dot(r, x))

def get_max_score(r, x):
    return -np.log(np.max(r))

def get_score(r, x):
    return get_this_score(r, x) -  get_max_score(r, x)

def descent_step(this_state, r, alpha):
    x = this_state
    # print("grad of r and x - ", get_grad(r, x))
    new_x = simplex_projection(x - alpha * get_grad(r, x))
    # print('new x - ', new_x)
    return new_x


def descent(begin_state, df_r,lr_scheduler=lambda i: 0.1):
    x = begin_state

    n = df_r.shape[0]

    scores = []
    vectors = [x]

    for i in range(n):
        r = df_r.iloc[i].values
        # print(x)
        score = get_score(r, x)
        scores.append(score)
        alpha = lr_scheduler(i)

        new_x = descent_step(x, r, alpha)

        x = new_x
        vectors.append(new_x)
        # print('r and x before next iteration - ', r, x)

    return (vectors, scores)
    

In [ ]:
data = load_data()
r = prepare_r(data)







best_alpha = 928

_, scores = descent(np.ones(r.shape[1]) * (1/r.shape[1]) , r, lr_scheduler=lambda x: 0)
